<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myProspects_Upside_Analysis_ATH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import datetime
import pytz
def get_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

get_time()
gen_output = 0

Run date time (IST): 2023-08-02 11:40:09


In [30]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

In [31]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365*5)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [32]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np

def read_file(path):
 df = pd.read_csv(path)
 return df

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Tradevesting/myProspectsScrips.csv'
df_mypf = read_file(path_mypf)

#mypf = mypf[mypf['Nifty100'] == 1]

def get_stock_list(df_mypf):
  stock_n100 = df_mypf['Symbol'].unique()
  exclude = ['CADILAHC','MMTC']
  stock_n100 = df_mypf[~df_mypf['Symbol'].isin(exclude) ]['Symbol'].unique()
  stock_ids = stock_n100
  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()
  return stock_ids

stock_ids = get_stock_list(df_mypf)

In [33]:
# star stocks & my stocks
star_stocks = list(df_mypf[df_mypf['StarStock'] == 1]['Stock'].values)
my_stocks = list(df_mypf[df_mypf['InPortfolio'] == 1]['Symbol'].values)
my_prospects = list(df_mypf[(df_mypf['LatestQtr'] == 1) & df_mypf['StarStock'] == 1]['Symbol'].values)

In [34]:
def print_distribution():

  # star stocks
  print('Overall - Star Stocks:')
  print(df_mypf['StarStock'].value_counts())
  print('-------')

  # v40/v40n stocks
  print('Overall - V40/V40N Stocks:')
  print(df_mypf['Criteria'].value_counts())
  print('-------')

  # latest quarter
  print('Overall - Latest Qtr:')
  print(df_mypf['LatestQtr'].value_counts())
  print('-------')

  # star + v40/v40n stocks
  print('Overall - Star + V40/V40N Stocks:')
  print(df_mypf[(df_mypf['Criteria'] == 'V40') | (df_mypf['Criteria'] == 'V40N')]['StarStock'].value_counts())
  print('-------')

  # stocks in portfolio
  print('Portfolio - Stocks:')
  print(df_mypf['InPortfolio'].value_counts())
  print('-------')

  # star stocks in portfolio
  print('Portfolio - Star Stocks:')
  print(df_mypf[(df_mypf['InPortfolio'] == 1)]['StarStock'].value_counts())
  print('-------')

  # latest quarter in portfolio
  print('Portfolio - Latest Quarter:')
  print(df_mypf[(df_mypf['InPortfolio'] == 1)]['LatestQtr'].value_counts())
  print('-------')

  # criteria of portfolio stocks
  print('Portfolio - Criteria:')
  print(df_mypf[(df_mypf['InPortfolio'] == 1)]['Criteria'].value_counts())
  print('-------')

  # strategies in portfolio stocks
  print('Portfolio - Strategy:')
  print(df_mypf[(df_mypf['InPortfolio'] == 1)]['Strategy'].value_counts())
  print('-------')

  # conviction distribution
  print('Portfolio - Conviction:')
  print(df_mypf[(df_mypf['InPortfolio'] == 1)]['Conviction'].value_counts())

print_distribution()

Overall - Star Stocks:
1.0    96
0.0    89
Name: StarStock, dtype: int64
-------
Overall - V40/V40N Stocks:
V40N    39
V40     38
VR      23
V200     6
AR       4
SS       1
Name: Criteria, dtype: int64
-------
Overall - Latest Qtr:
0    111
1     76
Name: LatestQtr, dtype: int64
-------
Overall - Star + V40/V40N Stocks:
1.0    43
0.0    34
Name: StarStock, dtype: int64
-------
Portfolio - Stocks:
0    135
1     52
Name: InPortfolio, dtype: int64
-------
Portfolio - Star Stocks:
1.0    33
0.0    19
Name: StarStock, dtype: int64
-------
Portfolio - Latest Quarter:
0    31
1    21
Name: LatestQtr, dtype: int64
-------
Portfolio - Criteria:
VR      23
V40N    11
V40     11
AR       4
V200     3
Name: Criteria, dtype: int64
-------
Portfolio - Strategy:
ATH     36
2T3Y    14
BTT      2
Name: Strategy, dtype: int64
-------
Portfolio - Conviction:
M    27
L    14
H    11
Name: Conviction, dtype: int64


In [35]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

In [36]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_features(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  stocks = ['INDIGOPNTS', 'IRCTC', 'GILLETTE', 'PAGEIND', 'BANDHANBNK', 'ABBOTINDIA', 'JCHAC',
              'NESTLEIND', 'PGHH', 'EQUITASBNK', 'SBICARD', 'ASTRAZEN', 'ICICIGI', 'RELAXO']

  if stock_name in stocks:
    actual_link = link_blueprint + stock_name.upper()
  else:
    actual_link = link_blueprint + stock_name.upper() + "/consolidated/"

  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:

    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
    current_roce = float(test_list[test_list.index('ROCE')+1:test_list.index('ROCE')+2][0])
    current_roe = float(test_list[test_list.index('ROE')+1:test_list.index('ROE')+2][0])
  except ValueError as ve:
    current_pe = 1000
    current_roce = 1000
    current_roe = 1000

  return current_pe, current_roce, current_roe

In [37]:
# function to compute stock attributes
def get_common_features():
  def stock_prec_dev(stock_symbol):
      #stock_symbol = 'ULTRACEMCO.NS'
      short_window = 20
      mid_window = 50
      long_window = 200
      moving_avg = 'SMA'

      # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
      start = datetime.datetime(*map(int, start_date.split('-')))
      end = datetime.datetime(*map(int, end_date.split('-')))
      stock_df = yfin.Ticker(stock_symbol).history(period='7y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
      stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
      stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
      stock_df.dropna(axis = 0, inplace = True) # remove any null rows

      stock_df['Stock'] = stock_symbol

      stock_df['Close'] = round(stock_df['Close'],2)
      stock_df['Max'] = round(max(stock_df['Close']),0)
      stock_df.ta.rsi(append=True)
      #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
      stock_df['Close'] = round(stock_df['Close'],0)
      stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
      stock_df.drop(['Open', 'Low', 'High', 'Volume'
                    ], axis=1, inplace=True)

      return stock_df

  df_prec_dev = pd.DataFrame()

  #tmp = stock_prec_dev('ULTRACEMCO.NS')
  #df_perform = df_perform.append(tmp)
  for stock_id in stock_ids:
      #print(stock_id)
      current_pe, current_roce, current_roe = get_current_features(stock_id)
      stock_id = stock_id.upper() + '.NS'
      tmp = stock_prec_dev(stock_id)
      tmp['Prev_Close'] = tmp['Close'].shift(1)
      tmp = tmp.reset_index()
      tmp = tmp.tail(1)
      tmp['Avg_Cost'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
      tmp['Shares'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]
      tmp['Med_PE'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['MedianPE'].values[0]
      tmp['Curr_PE'] = current_pe
      tmp['Curr_ROCE%'] = current_roce
      tmp['Curr_ROE%'] = current_roe
      tmp['Criteria'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Criteria'].values[0]
      tmp['Strategy'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Strategy'].values[0]
      tmp['Target'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Target'].values[0]
      tmp['LatestQtr'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['LatestQtr'].values[0]
      tmp['StarStock'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['StarStock'].values[0]
      tmp['Conviction'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Conviction'].values[0]
      df_prec_dev = df_prec_dev.append(tmp)

  return df_prec_dev, stock_prec_dev

df_common_features, stock_prec_dev = get_common_features()

In [38]:
# nifty in last 5 weeks
stock_prec_dev('^NSEI')[-21::5]

,Close,200_SMA,Dev%_200,Stock,Max,RSI_14
Date,,,,,,
2023-07-05 00:00:00+05:30,19398.0,17976.0,7.91,^NSEI,19979.0,76.0
2023-07-12 00:00:00+05:30,19384.0,18016.0,7.59,^NSEI,19979.0,68.0
2023-07-19 00:00:00+05:30,19833.0,18074.0,9.73,^NSEI,19979.0,78.0
2023-07-26 00:00:00+05:30,19778.0,18144.0,9.01,^NSEI,19979.0,67.0
2023-08-02 00:00:00+05:30,19539.0,18206.0,7.32,^NSEI,19979.0,53.0


In [39]:
stock_ids = df_common_features['Stock'].values
df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)

#df_common_features['Stock'] = df_common_features['Symbol'] + '.NS'
#df_prec_dev.drop(['Med_PE'], axis=1, inplace=True)

In [40]:
df_common_features['Discount%PE'] = round((df_common_features['Med_PE']-df_common_features['Curr_PE'])*100/(df_common_features['Med_PE']),0)
df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
df_common_features['Upside%Target'] = round((df_common_features['Target']-df_common_features['Close'])*100/(df_common_features['Close']),0)
df_common_features = df_common_features.set_index('Stock')

In [57]:
# portfolio stocks
cols = ['Close',	'Dev%_200',	'Upside%Target',	'RSI_14', 'Curr_ROCE%', 'Curr_ROE%', 'Conviction',  'LatestQtr', 'StarStock', 'Criteria', 'Strategy']
tmp_df = df_common_features[cols]
tmp_df = tmp_df[tmp_df.index.isin(df_mypf[df_mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=False)

qualified stocks: 52


,Close,Dev%_200,Upside%Target,RSI_14,Curr_ROCE%,Curr_ROE%,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,
GLAND.NS,1270.0,-8.51,207.0,66.0,14.80,10.90,L,0,0.0,VR,2T3Y
VALIANTORG.NS,547.0,1.30,183.0,59.0,14.80,13.00,L,0,0.0,VR,2T3Y
BANDHANBNK.NS,222.0,-5.30,170.0,46.0,6.67,11.90,M,1,0.0,VR,2T3Y
NIACL.NS,125.0,13.95,166.0,65.0,4.86,4.06,L,0,0.0,VR,ATH
LUXIND.NS,1595.0,6.17,163.0,62.0,12.70,9.95,M,0,0.0,V40N,2T3Y
VAIBHAVGBL.NS,349.0,10.55,151.0,70.0,11.10,9.02,L,0,0.0,VR,2T3Y
PNB.NS,60.0,17.94,150.0,53.0,4.11,3.34,M,1,1.0,VR,ATH
METROPOLIS.NS,1364.0,-0.14,127.0,40.0,17.40,15.20,L,0,0.0,VR,ATH
AAVAS.NS,1558.0,-9.77,97.0,54.0,9.50,14.10,M,0,1.0,VR,2T3Y


In [43]:
# top 5 from portfolio to book profit based on Upside%ATH
tmp_df.sort_values(by = 'Upside%Target', ascending=True).head(7)

,Close,Dev%_200,Upside%Target,RSI_14,Curr_ROCE%,Curr_ROE%,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,
PIDILITIND.NS,2617.0,4.12,6.0,50.0,24.6,18.7,H,0,1.0,V40,ATH
EICHERMOT.NS,3362.0,0.49,10.0,48.0,27.4,21.1,H,0,1.0,V40N,ATH
UJJIVANSFB.NS,49.0,63.17,12.0,77.0,1000.0,1000.0,M,1,1.0,V40N,ATH
TCS.NS,3439.0,5.93,13.0,59.0,59.1,46.9,H,1,1.0,V40,ATH
BATAINDIA.NS,1755.0,10.24,14.0,71.0,19.6,19.8,H,0,0.0,V40,ATH
BAJAJFINSV.NS,1544.0,4.28,17.0,40.0,12.7,14.8,H,1,1.0,V40,ATH
ICICIGI.NS,1372.0,16.70,17.0,58.0,21.2,17.6,H,1,1.0,V40,ATH


In [42]:
# top 10 stocks from portfolio for SIP based on 200 DMA and RSI
tmp_df[tmp_df['Dev%_200'] < 0].sort_values(by = 'RSI_14', ascending=True).head(10)

,Close,Dev%_200,Upside%Target,RSI_14,Curr_ROCE%,Curr_ROE%,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,
FINEORG.NS,4496.0,-12.02,56.0,35.0,65.40,49.4,M,1,0.0,V40N,ATH
KOTAKBANK.NS,1827.0,-0.98,24.0,39.0,6.86,14.2,L,1,1.0,V40,BTT
METROPOLIS.NS,1364.0,-0.14,127.0,40.0,17.40,15.2,L,0,0.0,VR,ATH
RAJESHEXPO.NS,512.0,-22.36,63.0,41.0,10.30,10.5,M,0,1.0,V40N,ATH
CLEAN.NS,1311.0,-8.94,91.0,42.0,44.50,33.2,L,0,1.0,VR,2T3Y
TEAMLEASE.NS,2325.0,-4.33,96.0,42.0,14.80,15.1,M,1,1.0,V40N,2T3Y
BANDHANBNK.NS,222.0,-5.30,170.0,46.0,6.67,11.9,M,1,0.0,VR,2T3Y
DMART.NS,3717.0,-0.38,37.0,48.0,20.10,16.0,M,1,1.0,AR,ATH
INFY.NS,1356.0,-3.62,33.0,51.0,40.70,31.8,H,1,1.0,V40,ATH


In [44]:
#df_common_features.drop([ '200_SMA', 'Avg_Cost', 'Shares', 'Med_PE', 'Curr_PE', 'Discount%PE'], axis=1, inplace=True)
tmp_base_df = df_common_features[~df_common_features.index.isin(df_mypf[df_mypf['InPortfolio'] == 1].Stock.values)][cols]

In [45]:
# latest quarter prospects (for upside)

tmp_df = tmp_base_df[tmp_base_df.index.isin(my_prospects)]
#tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
#tmp_df = tmp_df[(tmp_df['Upside%Target'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=False)

qualified stocks: 0


,Close,Dev%_200,Upside%Target,RSI_14,Curr_ROCE%,Curr_ROE%,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,


In [46]:
# star + v40/v40n prospects (for upside)

tmp_df = tmp_base_df[tmp_base_df.index.isin(star_stocks)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%Target'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=False)

qualified stocks: 7


,Close,Dev%_200,Upside%Target,RSI_14,Curr_ROCE%,Curr_ROE%,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,
DIXON.NS,4126.0,13.01,39.0,47.0,25.8,22.4,M,1,1.0,V40N,NaN
MOTILALOFS.NS,820.0,23.09,33.0,66.0,13.0,15.6,L,1,1.0,V40N,NaN
ASIANPAINT.NS,3349.0,10.28,27.0,46.0,35.2,27.7,H,1,1.0,V40,BTT
BAJFINANCE.NS,7178.0,9.55,27.0,41.0,11.8,23.5,H,1,1.0,V40,BTT
ANGELONE.NS,1492.0,10.39,26.0,39.0,44.0,47.1,M,1,1.0,V40N,NaN
ASTRAZEN.NS,3810.0,12.70,22.0,58.0,30.9,23.0,M,0,1.0,V40N,NaN
BERGEPAINT.NS,701.0,14.92,22.0,67.0,23.8,20.4,M,0,1.0,V40,NaN


In [47]:
# star + v40/v40n prospects (for breakout)

tmp_df = tmp_base_df[tmp_base_df.index.isin(star_stocks)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%Target'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=False)

qualified stocks: 9


,Close,Dev%_200,Upside%Target,RSI_14,Curr_ROCE%,Curr_ROE%,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,
CERA.NS,7734.0,23.29,4.0,56.0,25.80,19.4,H,0,1.0,V40N,NaN
MCDOWELL-N.NS,1000.0,18.01,4.0,61.0,19.90,17.5,L,1,1.0,V40N,NaN
POLYCAB.NS,4641.0,49.96,4.0,71.0,27.10,20.0,M,1,1.0,V40N,NaN
CAPLIPOINT.NS,922.0,27.47,3.0,73.0,26.80,22.4,M,0,1.0,V40N,NaN
MARICO.NS,557.0,9.28,3.0,59.0,43.00,35.8,M,1,1.0,V40,NaN
3MINDIA.NS,28818.0,21.46,1.0,63.0,31.40,23.4,M,0,1.0,V40N,NaN
BAJAJ-AUTO.NS,4863.0,25.64,1.0,59.0,26.20,20.2,L,1,1.0,V40,NaN
ICICIBANK.NS,984.0,8.65,1.0,59.0,6.82,18.7,H,1,1.0,V40,NaN
AKZOINDIA.NS,2831.0,23.78,0.0,75.0,34.70,26.1,M,0,1.0,V40,BTT


In [48]:
# star stocks (for upside)

tmp_df = tmp_base_df[tmp_base_df.index.isin(star_stocks)]
tmp_df = tmp_df[(tmp_df['Criteria'] != 'V40') & (tmp_df['Criteria'] != 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%Target'] >= 25)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=False)

qualified stocks: 6


,Close,Dev%_200,Upside%Target,RSI_14,Curr_ROCE%,Curr_ROE%,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,
ADANITRANS.NS,795.0,-51.38,416.0,50.0,10.50,11.60,L,0,1.0,NaN,NaN
ADANIGREEN.NS,1070.0,-18.32,178.0,62.0,7.81,22.80,L,0,1.0,NaN,NaN
ADANIENT.NS,2484.0,-7.52,68.0,61.0,10.10,9.63,M,0,1.0,NaN,NaN
GODREJIND.NS,480.0,7.22,34.0,47.0,9.75,11.20,L,0,1.0,NaN,NaN
UPL.NS,622.0,-12.77,34.0,30.0,15.70,13.40,L,0,1.0,NaN,NaN
SBICARD.NS,866.0,7.30,31.0,52.0,13.00,25.70,M,0,1.0,NaN,NaN


In [49]:
# star stocks (for breakout)

tmp_df = tmp_base_df[tmp_base_df.index.isin(star_stocks)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%Target'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
#print(tmp_df.index)
tmp_df.sort_values(by = 'Upside%Target', ascending=False)

qualified stocks: 9


,Close,Dev%_200,Upside%Target,RSI_14,Curr_ROCE%,Curr_ROE%,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,
CERA.NS,7734.0,23.29,4.0,56.0,25.80,19.4,H,0,1.0,V40N,NaN
MCDOWELL-N.NS,1000.0,18.01,4.0,61.0,19.90,17.5,L,1,1.0,V40N,NaN
POLYCAB.NS,4641.0,49.96,4.0,71.0,27.10,20.0,M,1,1.0,V40N,NaN
CAPLIPOINT.NS,922.0,27.47,3.0,73.0,26.80,22.4,M,0,1.0,V40N,NaN
MARICO.NS,557.0,9.28,3.0,59.0,43.00,35.8,M,1,1.0,V40,NaN
3MINDIA.NS,28818.0,21.46,1.0,63.0,31.40,23.4,M,0,1.0,V40N,NaN
BAJAJ-AUTO.NS,4863.0,25.64,1.0,59.0,26.20,20.2,L,1,1.0,V40,NaN
ICICIBANK.NS,984.0,8.65,1.0,59.0,6.82,18.7,H,1,1.0,V40,NaN
AKZOINDIA.NS,2831.0,23.78,0.0,75.0,34.70,26.1,M,0,1.0,V40,BTT


In [50]:
# stocks excluded from all time high

excluded_stocks = ['ACC.NS','AMBUJACEM.NS','CADILAHC.NS','GILLETTE.NS','NESTLEIND.NS','PGHH.NS', 'PVRINOX.NS', 'SIEMENS.NS']

tmp_df = tmp_base_df[tmp_base_df.index.isin(excluded_stocks)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=False)

qualified stocks: 2


,Close,Dev%_200,Upside%Target,RSI_14,Curr_ROCE%,Curr_ROE%,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,
PGHH.NS,15515.0,10.62,3.0,57.0,110.0,79.5,L,0,0.0,V40,NaN
NESTLEIND.NS,22682.0,10.94,2.0,50.0,138.0,108.0,H,0,0.0,V40,NaN


In [51]:
# stocks prospects

excluded_stocks = ['BANDHANBNK.NS','GICRE.NS','JMFINANCIL.NS','NIACL.NS','PNB.NS','UJJIVANSFB.NS']

tmp_df = tmp_base_df[tmp_base_df.index.isin(excluded_stocks)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=False)

qualified stocks: 0


,Close,Dev%_200,Upside%Target,RSI_14,Curr_ROCE%,Curr_ROE%,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,


In [52]:
# prospects with heavy upside
tmp_df = tmp_base_df[(tmp_base_df['Criteria'] == 'V40') | (tmp_base_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[tmp_df['Upside%Target'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=False)

qualified stocks: 8


,Close,Dev%_200,Upside%Target,RSI_14,Curr_ROCE%,Curr_ROE%,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,
JCHAC.NS,967.0,-13.52,233.0,34.0,1000.00,1000.00,M,1,0.0,V40N,NaN
WHIRLPOOL.NS,1451.0,1.91,84.0,53.0,8.87,6.37,M,0,0.0,V40,NaN
LALPATHLAB.NS,2318.0,7.46,81.0,51.0,18.50,15.00,M,1,0.0,V40N,NaN
SYMPHONY.NS,873.0,-7.23,68.0,46.0,14.80,13.50,M,1,0.0,V40N,NaN
SFL.NS,1180.0,-0.84,67.0,50.0,13.60,12.50,M,0,0.0,V40N,NaN
PFIZER.NS,3998.0,-0.14,50.0,69.0,27.20,19.80,L,0,0.0,V40,NaN
TATAELXSI.NS,7206.0,6.23,47.0,37.0,60.00,39.30,M,0,0.0,V40N,NaN
TTKPRESTIG.NS,788.0,1.58,47.0,64.0,18.20,13.80,M,0,0.0,V40N,NaN


In [53]:
# cross-check the features
tmp_df = tmp_base_df[(tmp_base_df['Criteria'] == 'V40') | (tmp_base_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[tmp_df['Upside%Target'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=False)

qualified stocks: 8


,Close,Dev%_200,Upside%Target,RSI_14,Curr_ROCE%,Curr_ROE%,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,
JCHAC.NS,967.0,-13.52,233.0,34.0,1000.00,1000.00,M,1,0.0,V40N,NaN
WHIRLPOOL.NS,1451.0,1.91,84.0,53.0,8.87,6.37,M,0,0.0,V40,NaN
LALPATHLAB.NS,2318.0,7.46,81.0,51.0,18.50,15.00,M,1,0.0,V40N,NaN
SYMPHONY.NS,873.0,-7.23,68.0,46.0,14.80,13.50,M,1,0.0,V40N,NaN
SFL.NS,1180.0,-0.84,67.0,50.0,13.60,12.50,M,0,0.0,V40N,NaN
PFIZER.NS,3998.0,-0.14,50.0,69.0,27.20,19.80,L,0,0.0,V40,NaN
TATAELXSI.NS,7206.0,6.23,47.0,37.0,60.00,39.30,M,0,0.0,V40N,NaN
TTKPRESTIG.NS,788.0,1.58,47.0,64.0,18.20,13.80,M,0,0.0,V40N,NaN


In [54]:
# analyse individual stock
stock_id = 'PVRINOX.NS'
df_common_features[df_common_features.index == stock_id]

,Close,200_SMA,Dev%_200,Max,RSI_14,Prev_Close,Avg_Cost,Shares,Med_PE,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,Target,LatestQtr,StarStock,Conviction,Discount%PE,Upside%Target
Stock,,,,,,,,,,,,,,,,,,,,
PVRINOX.NS,1599.0,1595.0,0.26,1917.0,76.0,1529.0,NaN,NaN,51.2,1000.0,1000.0,1000.0,NaN,NaN,1917.0,0,0.0,L,-1853.0,20.0


In [55]:
# save the output
if gen_output == 1:
  from google.colab import drive
  drive.mount('/content/drive')
  df_prec_dev = df_prec_dev.reset_index()
  df_prec_dev.to_csv('/content/drive/My Drive/data/stocks/myProspects-Upside-Analysis-ATH.csv', index=False)

In [56]:
# stock in last 5 weeks
#stock_ids = ['ICICIGI','DABUR','MOTILALOFS','JPPOWER','EQUITAS','GLAND','JMFINANCIL','LUXIND','UJJIVANSFB','VAIBHAVGBL','VALIANTORG']
stock_prec_dev('BERGEPAINT.NS')[-21::5]

,Close,200_SMA,Dev%_200,Stock,Max,RSI_14
Date,,,,,,
2023-07-05 00:00:00+05:30,677.0,605.0,11.90,BERGEPAINT.NS,858.0,62.0
2023-07-12 00:00:00+05:30,669.0,605.0,10.61,BERGEPAINT.NS,858.0,53.0
2023-07-19 00:00:00+05:30,684.0,607.0,12.61,BERGEPAINT.NS,858.0,62.0
2023-07-26 00:00:00+05:30,674.0,608.0,10.77,BERGEPAINT.NS,858.0,51.0
2023-08-02 00:00:00+05:30,702.0,610.0,15.13,BERGEPAINT.NS,858.0,68.0
